In [ ]:
import pandas as pd
import numpy as np
import unicodedata
import torch
from bs4 import BeautifulSoup
from pytorch_transformers import BertTokenizer, BertModel

import re

df = pd.read_csv('IMDB_Dataset.csv', encoding='utf-8')
# df = pd.read_csv('test.csv', encoding='utf-8')
# 시험삼아 해보세용..

In [ ]:
# Sentiment 1, 0으로 바꾸기
df['sentiment'] = (df['sentiment'] == 'positive').astype(int)

In [ ]:
# 문장 단위로 쪼개기
df_reviews = df.review.str.split('(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', expand=True)
df_reviews = df_reviews.stack().reset_index(level=1, drop=True).to_frame('review_single')
df_pre = df[["sentiment"]].merge(df_reviews, left_index=True, right_index=True, how='left')
df_reviews = df_pre.values

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
import pickle
import gzip

with gzip.open('Top5000_BtoA.pickle','rb') as f:
    Top5000_BtoA = pickle.load(f)

In [ ]:
# 전처리
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

def clean_review(raw_review: str) -> str:
    # 1. unicode to ASCII
    raw_review = unicodeToAscii(raw_review)
    
    # 2. Remove HTML
    review_text = BeautifulSoup(raw_review, "lxml").get_text()
    
    return review_text


def preprocess(review: str, total: int, show_progress: bool = True) -> list:
    if show_progress:
        global counter
        counter += 1
        if counter % 10000 == 0:
            print('Processing... %6i/%6i'% (counter, total))
          
    # 1. Clean text
    review = clean_review(review)
        
    # 2. BERT
    token_ids = [tokenizer.encode(review, add_special_tokens=True)]
    
    L = len(token_ids[0])

    # 길이가 길면 그 문장을 넘어감.
    if L >= 100:
        return None
    
    # TOP5000에 없는 단어가 있는 문장이면 넘어감.
    for word in token_ids[0]:
        try:
            Top5000_BtoA[word]
        except:
            return None
        
    return Top5000_BtoA[token_ids[0][1]]

In [ ]:
counter = 0

first = {} # 분포

cur_size = 0
batch_size = 512
num = 1

df_length = len(df_reviews)
for df_review in df_reviews:
    x = preprocess(df_review[1], df_length)
    if x == None:
        continue
    try:
        first[x] += 1
    except:
        first[x] = 1

In [ ]:
first_list = [] # 누적분포
acc = 0

for key in first.keys():
    first_list.append((key, acc))
    acc += first[key]

first_list.append((-1, acc))

In [3]:
import pickle
import gzip

with gzip.open('Top5000_first_list.pickle','rb') as f:
    first_list = pickle.load(f)

In [4]:
def random_number(first_list):
    import random
    r = random.randrange(first_list[-1][1])
    for i in range(len(first_list)):
        if(first_list[i][1] <= r and r < first_list[i+1][1]):
            return first_list[i][0]

In [7]:
random_number(first_list)

89